<a href="https://colab.research.google.com/github/anuraglahon16/ResearchPaperSimplify/blob/main/FinalProjectAIMcode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install dependencies
!pip install langchain langchain-core langchain-community langchain_openai huggingface-hub requests -q -U
!pip install arxiv pymupdf faiss-cpu -q -U
!pip install chromadb -q
!pip install tenacity -q
!pip install duckduckgo-search -q
!pip install scholarly -q
!pip install xmltodict

# Set up environment variables
import os
import getpass
from tenacity import retry, stop_after_attempt, wait_exponential

OPENAI_API_KEY = getpass.getpass("OpenAI API Key:")

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.document_loaders import ArxivLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA, ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.agents import initialize_agent, Tool
from langchain.utilities import DuckDuckGoSearchAPIWrapper, PubMedAPIWrapper, ArxivAPIWrapper

# Get user input for the paper to search
search_query = input("Enter the topic or keywords to search for papers: ")

# Load scientific papers from arXiv
loader = ArxivLoader(query=search_query, load_max_docs=10)
files = loader.load()

# Split documents into chunks
text_splitter = CharacterTextSplitter(separator='\n---\n', chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(files)

# Create vector database and store papers
embedding_model = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY, model="text-embedding-3-large")
db = Chroma.from_documents(docs, embedding_model, persist_directory='./chroma_db')

# Initialize the language model
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name="gpt-3.5-turbo", temperature=0.5, max_tokens=150)

# Initialize the conversation memory
chat_memory = ConversationBufferMemory(ai_prefix="AI Assistant")

# Define the conversation template
template = """
You're an AI assistant and your task is to gather all details from a user who wants to understand a scientific concept.

At the beginning, shortly describe the purpose of this conversation.

You should gather answers for the following questions:

- What specific questions do you have about this topic?
- What is your current level of understanding of this topic?
- Are there any related concepts you'd like me to explain as well?

Don't answer the question you are asking.

Be patient and encouraging if the user doesn't know how to answer some questions, and help guide them.

Ask one question at a time.

Once you have gathered all the details, thank the user for their responses, summarize the relevant information that will help you provide the best explanation, and put '<END-OF-CONVERSATION>'

Current conversation:
{history}
Human: {input}
AI assistant:
"""

# Create the conversation prompt
prompt = PromptTemplate(input_variables=["history", "input"], template=template)
conversation = ConversationChain(
    prompt=prompt,
    llm=llm,
    verbose=False,
    memory=chat_memory
)

# Start the conversation
current_input = f"I want to learn about {search_query}"

end_seq = '<END-OF-CONVERSATION>'
user_requirements = ''

while True:
    ai_response = conversation.predict(input=current_input)
    print(ai_response)

    if end_seq in ai_response:
        user_requirements = chat_memory.chat_memory.messages[-1].content.replace(end_seq, '')
        break

    user_input = input('User: ')
    current_input = user_input

# Initialize the DuckDuckGo search tool
search = DuckDuckGoSearchAPIWrapper()

# Initialize the RAG tool
rag_tool = Tool(
    name="RAG System",
    func=lambda q: RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=db.as_retriever()).run(q),
    description="Useful for answering questions based on the retrieved papers."
)

# Initialize the PubMed search tool
pubmed_search = PubMedAPIWrapper()

# Initialize the arXiv search tool
arxiv_search = ArxivAPIWrapper()

tool_belt = [
    Tool(
        name="DuckDuckGo Search",
        func=search.run,
        description="Useful for searching for additional information on the internet."
    ),
    rag_tool,
    Tool(
        name="PubMed Search",
        func=pubmed_search.run,
        description="Useful for searching for biomedical literature on PubMed."
    ),
    Tool(
        name="arXiv Search",
        func=arxiv_search.run,
        description="Useful for searching for papers on arXiv."
    )
]

# Generate the answer using the agent with retry logic
@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=4, max=10))
def generate_answer(query):
    agent = initialize_agent(tool_belt, llm, agent="zero-shot-react-description", verbose=True)
    return agent.run(f"Use the available tools to answer the query: {query}. If one tool doesn't provide a satisfactory answer, try using the other tools.")

try:
    answer = generate_answer(user_requirements)
except Exception as e:
    print(f"Error generating answer: {str(e)}")
    answer = "I apologize, I encountered an error while generating the answer. I tried using the available tools, but I still don't have enough relevant information to provide a satisfactory explanation. I would recommend searching for more specific papers or resources related to the topic to get a better understanding."

# Retrieve similar papers
similar_papers = db.similarity_search(user_requirements, k=3)

if similar_papers:
    # Generate a prompt for recommending similar papers
    prompt_template = PromptTemplate(
        input_variables=["papers"],
        template="""
        Based on the user's requirements and the retrieved papers, recommend 3 similar papers that could help the user better understand the topic.

        Retrieved Papers:
        {papers}
        """
    )

    # Format the similar papers into a string
    similar_papers_str = "\n".join([f"- {paper.metadata.get('title', 'Title not available')}" for paper in similar_papers])

    # Create the prompt for recommending similar papers
    recommend_prompt = prompt_template.format(papers=similar_papers_str)

    # Generate the recommendation using the language model
    recommendation = llm.invoke(recommend_prompt)
else:
    recommendation = "I apologize, I couldn't find any similar papers based on the user's requirements. The loaded papers do not seem to contain enough relevant information about the topic. I would suggest searching for more specific papers related to the topic to get a better understanding."

# Print the answer and recommendation
print("Answer:")
print(answer)
print("\nRecommended Papers:")
print(recommendation)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.5/810.5 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.1/269.1 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.6/388.6 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.9/262.9 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


The purpose of this conversation is to gather all the necessary details from you in order to help you understand the scientific concept of Lora fine tuning.

What specific questions do you have about this topic?
User: How to fine tune using Lora
What is your current level of understanding of this topic?
User: novice
Are there any related concepts you'd like me to explain as well?
User: Compare Lora and QLora
Thank you for your responses. To summarize, you are looking to understand how to fine-tune using Lora, you are a novice in this topic, and you would like to compare Lora and QLora. I will provide you with explanations that will help you understand Lora fine tuning. <END-OF-CONVERSATION>


> Entering new AgentExecutor chain...


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


I need to gather information on fine-tuning using Lora and compare it with QLora.
Action: DuckDuckGo Search
Action Input: "fine-tuning using Lora"
Observation: Fine-Tuning, LoRA and QLoRA. In the realm of language models, fine tuning an existing language model to perform a specific task on specific data is a common practice. This involves adding a task-specific head, if necessary, and updating the weights of the neural network through backpropagation during the training process. It is important to ... Fine-Tuning: Fine-tuning a model refers to the process of taking a pre-trained model (model trained on some big, public corpus) and further training it on a new, smaller dataset or with a specific ... November 24th, 2023. Low-Rank Adaptation (LoRA) method is a fine-tuning method introduced by a team of Microsoft researchers in 2021. Since then, it has become a very popular approach to fine-tuning large language models, diffusion models (such as for image-generation), and other types of AI